# **3118. Friday Purchase III** 

**Table: Purchases**
``` sql
+---------------+------+
| Column Name   | Type |
+---------------+------+
| user_id       | int  |
| purchase_date | date |
| amount_spend  | int  |
+---------------+------+
```
(user_id, purchase_date, amount_spend) is the primary key (combination of columns with unique values) for this table.

purchase_date will range from November 1, 2023, to November 30, 2023, inclusive of both dates.

Each row contains user_id, purchase_date, and amount_spend.

**Table: Users**
``` sql
+-------------+------+
| Column Name | Type |
+-------------+------+
| user_id     | int  |
| membership  | enum |
+-------------+------+
```
user_id is the primary key for this table.

membership is an ENUM (category) type of ('Standard', 'Premium', 'VIP').

Each row of this table indicates the user_id, membership type.

Write a solution to calculate the total spending by Premium and VIP members on each Friday of every week in November 2023.  If there are no purchases on a particular Friday by Premium or VIP members, it should be considered as 0.

Return the result table ordered by week of the month,  and membership in ascending order.

The result format is in the following example.

 

**Example:**

**Input:**

**Purchases table:**
``` sql
+---------+---------------+--------------+
| user_id | purchase_date | amount_spend |
+---------+---------------+--------------+
| 11      | 2023-11-03    | 1126         |
| 15      | 2023-11-10    | 7473         |
| 17      | 2023-11-17    | 2414         |
| 12      | 2023-11-24    | 9692         |
| 8       | 2023-11-24    | 5117         |
| 1       | 2023-11-24    | 5241         |
| 10      | 2023-11-22    | 8266         |
| 13      | 2023-11-21    | 12000        |
+---------+---------------+--------------+
```
**Users table:**
``` sql
+---------+------------+
| user_id | membership |
+---------+------------+
| 11      | Premium    |
| 15      | VIP        |
| 17      | Standard   |
| 12      | VIP        |
| 8       | Premium    |
| 1       | VIP        |
| 10      | Standard   |
| 13      | Premium    |
+---------+------------+
```
**Output:**
``` sql
+---------------+-------------+--------------+
| week_of_month | membership  | total_amount |
+---------------+-------------+--------------+
| 1             | Premium     | 1126         |
| 1             | VIP         | 0            |
| 2             | Premium     | 0            |
| 2             | VIP         | 7473         |
| 3             | Premium     | 0            |
| 3             | VIP         | 0            |
| 4             | Premium     | 5117         |
| 4             | VIP         | 14933        |
+---------------+-------------+--------------+
```      
**Explanation:**

During the first week of November 2023, a transaction occurred on Friday, 2023-11-03, by a Premium member amounting to $1,126. No transactions were made by VIP members on this day, resulting in a value of 0.


For the second week of November 2023, there was a transaction on Friday, 2023-11-10, and it was made by a VIP member, amounting to $7,473. Since there were no purchases by Premium members that Friday, the output shows 0 for Premium members.


Similarly, during the third week of November 2023, no transactions by Premium or VIP members occurred on Friday, 2023-11-17, which shows 0 for both categories in this week.


In the fourth week of November 2023, transactions occurred on Friday, 2023-11-24, involving one Premium member purchase of $5,117 and VIP member purchases totaling $14,933 ($9,692 from one and $5,241 from another).


Note: The output table is ordered by week_of_month and membership in ascending order.

**Solution-1:**

``` sql
with cte as(
SELECT ceil(day(purchase_date)/7) as week_of_month, u.membership,
SUM(amount_spend) AS total_amount
FROM Users u
JOIN Purchases p 
ON u.user_id = p.user_id and DAYNAME(purchase_date) = 'FriDay' 
and date_format(purchase_date,'%Y-%m') = '2023-11' and u.membership in ('VIP', 'Premium')
group by 1,2)

,cte2 as(
SELECT 2 AS week_of_month, 'Premium' AS membership
UNION ALL
SELECT 3 AS week_of_month, 'Premium' AS membership
UNION ALL
SELECT 4 AS week_of_month, 'Premium' AS membership
UNION ALL
SELECT 1 AS week_of_month, 'Premium' AS membership
UNION ALL
SELECT 1 AS week_of_month, 'VIP' AS membership
UNION ALL
SELECT 2 AS week_of_month, 'VIP' AS membership
UNION ALL
SELECT 3 AS week_of_month, 'VIP' AS membership
UNION ALL
SELECT 4 AS week_of_month, 'VIP' AS membership
)

select cte2.week_of_month, cte2.membership, coalesce(cte.total_amount,0) as total_amount
from cte2
left join cte
on cte2.week_of_month = cte.week_of_month and cte2.membership = cte.membership
order by cte2.week_of_month, cte2.membership

**Solution-2:**

``` sql
WITH week_membership_combinations AS (
SELECT *
FROM (
    SELECT 1 AS week_of_month UNION ALL
    SELECT 2 UNION ALL
    SELECT 3 UNION ALL
    SELECT 4
) AS weeks
CROSS JOIN (
    SELECT 'Premium' AS membership UNION ALL
    SELECT 'VIP'
) AS types
),

amount_per_membership AS (
SELECT SUM(amount_spend) AS total_amount_base,
       u.membership,
       FLOOR((DAYOFMONTH(p.purchase_date) - 1) / 7) + 1 AS week_of_month
FROM Purchases AS p
JOIN Users AS u USING(user_id)
WHERE DAYOFWEEK(p.purchase_date) = 6
GROUP BY week_of_month, membership)

SELECT wc.week_of_month, 
       wc.membership,
       IFNULL(am.total_amount_base, 0) AS total_amount
FROM week_membership_combinations AS wc 
LEFT JOIN  amount_per_membership AS am
ON am.week_of_month = wc.week_of_month AND am.membership = wc.membership
ORDER BY week_of_month, membership;